In [9]:
# building an autoencoder from scratch
# use case: anomaly detection for credit card transactions
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset

In [5]:
# working with the data
df = pd.read_csv('dataset/creditcard.csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
# --- split ---
shuffled_df = df.sample(frac=1, random_state=42).reset_index(drop=True)
n = len(shuffled_df)
train_end = int(0.7 * n)
val_end = int(0.8 * n)

train_df = shuffled_df.iloc[:train_end].copy()
val_df   = shuffled_df.iloc[train_end:val_end].copy()
test_df  = shuffled_df.iloc[val_end:].copy()

# --- train on NORMALS only ---
train_df = train_df[train_df["Class"] == 0].reset_index(drop=True)

# --- features to use ---
features = ['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14',
            'V15','V16','V17','V18','V19','V20','V21','V22','V23','V24','V25','V26','V27','V28','Amount']

# stabilize Amount then standardize
train_df.loc[:, "Amount"] = np.log1p(train_df["Amount"])
val_df.loc[:,   "Amount"] = np.log1p(val_df["Amount"])
test_df.loc[:,  "Amount"] = np.log1p(test_df["Amount"])

# --- fit scaling on TRAIN NORMALS ONLY ---
train_mean = train_df[features].mean()
train_std  = train_df[features].std().replace(0, 1)  # avoid div by 0

def apply_standardize(df_):
    df_.loc[:, features] = (df_[features] - train_mean) / train_std
    return df_

train_df = apply_standardize(train_df)
val_df   = apply_standardize(val_df)
test_df  = apply_standardize(test_df)

# --- PyTorch Dataset(s) ---
# here's the cool thing about autoencoders: the dataset must return the input and the target but for training, the input is the same as the target since the autoencoder is trying to reconstruct the input
class CreditCardDataset(Dataset):
    def __init__(self, df, feature_cols):
        # convert to numpy array
        X = df[feature_cols].values.astype(np.float32)
        # convert to tensor
        self.X = torch.from_numpy(X)

    def __len__(self):
        # return the number of rows in the dataframe
        return self.X.shape[0]

    def __getitem__(self, idx):
        # return the row at the given index
        x = self.X[idx]
        return x, x  # input, target

class CreditCardEvalDataset(Dataset):
    def __init__(self, df, feature_cols):
        X = df[feature_cols].values.astype(np.float32)
        y = df["Class"].values.astype(np.int64)
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_ds = CreditCardDataset(train_df, features)
val_ds   = CreditCardEvalDataset(val_df, features)
test_ds  = CreditCardEvalDataset(test_df, features)

# dataloader
train_dl = DataLoader(train_ds, batch_size=128, shuffle=True)
val_dl   = DataLoader(val_ds, batch_size=128, shuffle=False)
test_dl  = DataLoader(test_ds, batch_size=128, shuffle=False)

# testing the dataloader
for i, (x, y) in enumerate(train_dl):
    print(x.shape, y.shape)
    if i == 0:   # stop after 10 batches
        break

torch.Size([128, 29]) torch.Size([128, 29])


In [ ]:
# building an autoencoder from scratch
# each successive layer reduces the dimensionality, forming a funnel-like shape.
# input → 128 → 64 → 32 → 16 → Latent (bottleneck)
class Encoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(64),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(32),
            nn.Linear(32, latent_dim),
        )
        
    def forward(self, x):
        return self.layers(x)
    
class Decoder(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(latent_dim, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(32),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(64),
            nn.Linear(64, output_dim),
        )

    def forward(self, x):
        return self.layers(x)
    
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super().__init__()
        self.encoder = Encoder(input_dim, latent_dim)
        self.decoder = Decoder(latent_dim, input_dim)
        
    def forward(self, x):
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat